### Notebook: Previsão de vitória no LoL
Importar bibliotecas

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

### Carregar o Dataset

In [6]:
df = pd.read_csv("../../datasets/Challenger_Ranked_Games.csv")

### Selecionar features “não óbvias”

In [7]:
continuous_features = [
    'gameDuraton',
    'blueWardPlaced', 'redWardPlaced',
    'blueWardkills', 'redWardkills',
    'blueTotalLevel', 'blueAvgLevel', 'redTotalLevel', 'redAvgLevel',
    'blueJungleMinionKills', 'redJungleMinionKills',
    'blueKillingSpree', 'redKillingSpree',
    'blueTotalHeal', 'redTotalHeal'
]

binary_features = [
    'blueFirstBlood', 'redFirstBlood',
    'blueFirstTower', 'redFirstTower',
    'blueFirstDragon', 'redFirstDragon',
    'blueFirstBaron', 'redFirstBaron',
    'blueFirstInhibitor', 'redFirstInhibitor'
]

# Escalonar features contínuas entre 0 e 1
scaler = MinMaxScaler()
X_cont = scaler.fit_transform(df[continuous_features])

# Concatenar com features binárias
X = np.hstack([X_cont, df[binary_features].values])

# Target
y = df['blueWins']

### Configurar 5-fold Cross Validation

In [8]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Inicializar classificadores

In [9]:
rf = RandomForestClassifier(random_state=42)
svm = SVC()
lr = LogisticRegression(max_iter=5000)  # max_iter aumentado para convergência

### Avaliar classificadores com 5-fold CV

In [10]:
scores_rf = cross_val_score(rf, X, y, cv=kf, scoring='accuracy')
scores_svm = cross_val_score(svm, X, y, cv=kf, scoring='accuracy')
scores_lr = cross_val_score(lr, X, y, cv=kf, scoring='accuracy')

print("Random Forest Accuracy: %.4f ± %.4f" % (scores_rf.mean(), scores_rf.std()))
print("SVM Accuracy: %.4f ± %.4f" % (scores_svm.mean(), scores_svm.std()))
print("Logistic Regression Accuracy: %.4f ± %.4f" % (scores_lr.mean(), scores_lr.std()))

Random Forest Accuracy: 0.9702 ± 0.0028
SVM Accuracy: 0.9543 ± 0.0028
Logistic Regression Accuracy: 0.9645 ± 0.0030


### Ajuste de hiperparâmetros (GridSearch)

In [11]:
param_rf = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_rf, cv=kf, scoring='accuracy')
grid_rf.fit(X, y)

param_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_svm = GridSearchCV(SVC(), param_svm, cv=kf, scoring='accuracy')
grid_svm.fit(X, y)

print("\n=== Melhor Random Forest ===")
print("Parâmetros:", grid_rf.best_params_)
print("Acurácia média 5-fold:", grid_rf.best_score_)

print("\n=== Melhor SVM ===")
print("Parâmetros:", grid_svm.best_params_)
print("Acurácia média 5-fold:", grid_svm.best_score_)


=== Melhor Random Forest ===
Parâmetros: {'max_depth': 20, 'n_estimators': 200}
Acurácia média 5-fold: 0.9707107135183757

=== Melhor SVM ===
Parâmetros: {'C': 10, 'kernel': 'linear'}
Acurácia média 5-fold: 0.9749108697889932


### Explicação dos resultados

In [12]:
"""
1. Random Forest: bom desempenho em relações não-lineares e interações entre features. Possui feature_importances_ para análise interpretativa.
2. SVM: sensível à escala das features; aqui usamos MinMaxScaler para melhorar a convergência e performance.
3. Logistic Regression: baseline linear; após escalonamento e aumento de max_iter, convergiu e apresenta resultados estáveis.

    Escolha do modelo final:
Com base na acurácia média 5-fold, desvio padrão e interpretabilidade,
o Random Forest costuma ser a melhor opção para este tipo de problema.
"""

'\n1. Random Forest: bom desempenho em relações não-lineares e interações entre features. Possui feature_importances_ para análise interpretativa.\n2. SVM: sensível à escala das features; aqui usamos MinMaxScaler para melhorar a convergência e performance.\n3. Logistic Regression: baseline linear; após escalonamento e aumento de max_iter, convergiu e apresenta resultados estáveis.\n\n    Escolha do modelo final:\nCom base na acurácia média 5-fold, desvio padrão e interpretabilidade,\no Random Forest costuma ser a melhor opção para este tipo de problema.\n'